In [1]:
import pickle
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score

# Veri setini yükleme

In [2]:
data=pd.read_csv('derlemler/filtrelenmis_temizlenmis_derlem.csv.gz')
data.head()

,news_title,summary,category,date,link,news,category_backup
0,mili piyango yılbaşı çekilişi sıralı tam liste...,her yıl büyük bir heyecana sahne olan ve milyo...,Gündem,2019-12-31,/gundem/mili-piyango-yilbasi-cekilisi-sirali-t...,i̇nsanların en büyük hayallerinden biri zengin...,Gündem
1,mili piyango sıralı tam listesi aralık çekiliş...,her yıl büyük bir heyecana sahne olan milli pi...,Gündem,2019-12-31,/gundem/mili-piyango-sirali-tam-listesi-31-ara...,i̇nsanların en büyük hayallerinden biri zengin...,Gündem
2,mpi̇ bilet amorti ikramiye sonuç sorgulama ekr...,milli piyango yılbaşı özel çekilişi aralık tar...,Gündem,2020-01-01,/gundem/mpi-bilet-amorti-ikramiye-sonuc-sorgul...,milli piyango yılbaşı özel çekilişinin ardında...,Gündem
3,yılbaşı mesajları ve sözleri sevdiklerinize gö...,google yılbaşı gününe özel doodle yayımladı yı...,Gündem,2019-12-31,/gundem/yilbasi-mesajlari-sosyal-medya-ve-tele...,yeni yıl heyecanı tüm yurdumuzu sardı bu akşam...,Gündem
4,yeni yıl mesajları yılbaşı mesajları kısa uzun...,bugün günlerden aralık senenin son günü yılbaş...,Gündem,2019-12-31,/gundem/yilbasi-kutlama-mesajlari-2020-yeni-yi...,bugün en özel günlerden biri yeni yıl öncesi k...,Gündem


In [3]:
print(f"Toplam örnek sayısı: {len(data)}")

Toplam örnek sayısı: 81407


In [4]:
print("Haber türüne göre örnek sayıları:")
for cat in data.category.unique():
    print(f"{cat}: {len(data[data['category']==cat])}")

Haber türüne göre örnek sayıları:
Gündem: 21031
Ekonomi: 11165
Spor: 10699
Siyaset: 14530
Dünya: 4792
Yaşam: 1696
Pazar: 2090
Ege: 6131
Magazin: 3980
Kültür_Sanat: 1265
Teknoloji_Bilim: 3105
Cumartesi: 923


### Çeşitli kategorilerdeki haberlerin çıkartılması

In [5]:
cat_drop = ['Gündem', 'Pazar', 'Cumartesi']

In [6]:
data=data.drop(data[data.category.apply(lambda x: True if x in cat_drop else False)].index, axis=0)
data=data.reset_index(drop=True)
print(f"Geriye kalan örnek sayısı: {len(data)}")

Geriye kalan örnek sayısı: 57363


### Stop wordlerin belirlenmesi
Milliyet de bir stop word olarak belirlenmiştir ve listeye eklenmiştir. (çoğu haberde var)

In [7]:
stop_words_tr = pd.read_csv('stop_words_tr.txt', sep="\n", header=None)
stop_words_tr=stop_words_tr.transpose().values.tolist()[0]
stop_words_tr.append('milliyet')

In [8]:
# Yedek veri oluşturma
df=data.copy()

### Bazı kategorideki haberlerin birleştirilmesi ve çıkarılması
Geriye kalanlar:

5-kategori: Ekonomi, Siyaset, Spor, Teknoloji-Bilim, Diğer (Kültür-Sanat, Magazin, Yaşam)

In [9]:
df=df.drop(df[df.category=='Ege'].index)
df=df.drop(df[df.category=='Dünya'].index)
df['category'] = df['category'].apply(lambda x: 'Diğer' if x in ['Yaşam', 'Magazin', 'Kültür_Sanat'] else x)
print(f"Geriye kalan örnek sayısı: {len(df)}")

Geriye kalan örnek sayısı: 46440


In [10]:
data_titles = df.news_title
data_summaries = df.summary
data_news = df.news
data_labels = df.category
#data_labels = data_labels.astype('category').cat.codes

le = LabelEncoder()
data_labels = le.fit(data_labels).transform(data_labels)
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

print(le_name_mapping)

{'Diğer': 0, 'Ekonomi': 1, 'Siyaset': 2, 'Spor': 3, 'Teknoloji_Bilim': 4}


### Verinin %70 train (eğitim), %30 test (deney) olarak ayrılması

In [11]:
train_X, test_X, train_Y, test_Y= train_test_split(data_news, data_labels, stratify=data_labels,test_size=0.30, shuffle=True)

### Haber metinlerinin vektöre dönüşümü

In [12]:
#token_pat= '[a-z]{3,}'   ngram_range=(1,2)
cv = CountVectorizer(max_df=0.80, min_df=2, ngram_range= (1, 1), stop_words=stop_words_tr) #token_pattern=token_pat , stop_words=stop_words_tr
train_data = cv.fit_transform(train_X)

sum_words = train_data.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)

reverse_vocabulary = {}
vocabulary = cv.vocabulary_
for word in vocabulary:
    index = vocabulary[word]
    reverse_vocabulary[index] = word

train_data_vocab=cv.get_feature_names()
print (len(train_data_vocab))

223078


### Haberlerde en çok ve en az geçen 10 kelimeler

In [13]:
words_freq[:10]

[('türkiye', 51290),
 ('ın', 42307),
 ('in', 36373),
 ('büyük', 27271),
 ('yeni', 24340),
 ('son', 21332),
 ('iyi', 20413),
 ('türk', 20245),
 ('ilk', 19980),
 ('yüzde', 19641)]

In [14]:
words_freq[-10:]

[('ödetilecek', 2),
 ('arayanlarla', 2),
 ('geliyori', 2),
 ('müvekkiline', 2),
 ('kotanjant', 2),
 ('mücevherinin', 2),
 ('ulas', 2),
 ('madi', 2),
 ('yaklaşımınızın', 2),
 ('peşindekilere', 2)]

In [15]:
cv2 = CountVectorizer(vocabulary=train_data_vocab)
#cv2 = TfidfVectorizer(vocabulary=train_data_vocab)
test_data = cv2.fit_transform(test_X)
test_data_vocab=cv2.get_feature_names()
print(len(test_data_vocab))

223078


### Model eğitimi 

In [16]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 1)
clf.fit(train_data, train_Y)
y_pred = clf.predict(test_data)
print(accuracy_score(test_Y, y_pred))

0.8079959804766006


### Sonuçlar

In [17]:
print(confusion_matrix(test_Y, y_pred))

[[1634  150  202   72   24]
 [ 233 2369  633   91   23]
 [ 347  416 3473  108   15]
 [  71   57  160 2918    4]
 [  38   25    6    0  863]]


In [18]:
print(classification_report(test_Y, y_pred, target_names=le.classes_))

                 precision    recall  f1-score   support

          Diğer       0.70      0.78      0.74      2082
        Ekonomi       0.79      0.71      0.74      3349
        Siyaset       0.78      0.80      0.79      4359
           Spor       0.92      0.91      0.91      3210
Teknoloji_Bilim       0.93      0.93      0.93       932

       accuracy                           0.81     13932
      macro avg       0.82      0.82      0.82     13932
   weighted avg       0.81      0.81      0.81     13932



### Kategorilere göre en çok geçen kelimeler

In [19]:
coefs = clf.coef_
target_names = le.classes_

for i in range(len(target_names)):
    words = []
    for j in coefs[i].argsort()[-15:]:
        words.append(reverse_vocabulary[j])
    print (target_names[i], '-', words, "\n")

Diğer - ['yer', 'gün', 'türk', 'zaman', 'iyi', 'son', 'kitap', 'yıl', 'stanbul', 'ilk', 'büyük', 'yeni', 'türkiye', 'in', 'ın'] 

Ekonomi - ['tr', 'com', 'abd', 'son', 'başkanı', 'önemli', 'türk', 'dolar', 'yeni', 'ın', 'yıl', 'in', 'büyük', 'yüzde', 'türkiye'] 

Siyaset - ['yok', 'dedi', 'büyük', 'türk', 'yeni', 'ankara', 'parti', 'başbakan', 'başkanı', 'erdoğan', 'chp', 'genel', 'in', 'ın', 'türkiye'] 

Spor - ['teknik', 'yok', 'milli', 'son', 'ilk', 'türkiye', 'takım', 'futbol', 'büyük', 'beşiktaş', 'iyi', 'in', 'galatasaray', 'ın', 'fenerbahçe'] 

Teknoloji_Bilim - ['oyun', 'büyük', 'iyi', 'samsung', 'binay', 'galaxy', 'onur', 'sahip', 'in', 'ın', 'apple', 'tr', 'com', 'yeni', 'phone'] 

